### 1. 상품 테이블 데이터 삽입

In [3]:
import requests
from bs4 import BeautifulSoup

product_category = {
	'saleandevent' : '세일&이벤트',
	'giftset' : '선물세트',
	'number' : '넘버 시리즈',
	'express' : '익스프레스 시리즈',
    'accessory' : '액세서리'
}

head = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}

In [4]:
# 상품명, 원가, 할인율, 상품설명, 원산지, 구매혜택 포인트, 배송방법, 배송비  
product_data = []
cnt = 1
for i in product_category.keys():
    url = f'https://saturnbird.co.kr/{i}'
    soup = BeautifulSoup(requests.get(url, headers=head).text)
    for_link = soup.find_all('div', class_='item-thumbs')

    product_list = []
    for link in for_link:
        product_list.append('https://saturnbird.co.kr' + link.find('a')['href'])

    for product_link in product_list:
        soup = BeautifulSoup(requests.get(product_link, headers=head).text)
        product_info = soup.find('div', class_='goods_form')

        product_name = (
            product_info
            .find('div', class_='view_tit no-margin-top title_font_style tw-flex tw-flex-wrap tw-gap-[8px] tw-items-center')
            .get_text()
            .replace('SALE', '')
            .replace('SOLDOUT', '')
            .replace('NEW', '')
        )

        sale_price = int(
            product_info.find('span').get_text().strip().replace(',', '')[:-1]
        )

        try:
            sale_percentage = int(
                product_info.find('span', class_='sale_percentage').get_text()[:-1]
            )
        except:
            sale_percentage = 0

        product_summary = (
            product_info
            .find('div', class_='goods_summary body_font_color_70')
            .get_text()
            .replace('\n', '')
            .strip()
        )

        info = {
            '상품ID' : cnt,
            '상품분류': product_category[i],
            '상품명': product_name,
            '원가': sale_price,
            '할인율': sale_percentage,
            '상품설명': product_summary
        }

        url2 = 'https://saturnbird.co.kr/shop/prod_detail_make.cm?idx=45&change_country=KR&__=8d91239afef0103e7f41520c216625520b270d3d'
        detail_html = BeautifulSoup(requests.get(url2, headers=head).json()['html'])

        for opt in detail_html.find_all('div', class_='option_wrap'):
            topic_info = opt.find('span', class_='option_title text-bold').get_text()

            if topic_info in ['원산지', '구매혜택', '배송 방법', '배송비']:
                detail_value = opt.find('span', class_='option_data').get_text().strip().split(' ')
                try:
                    info[topic_info] = int(detail_value[0].replace('원', '').replace(',', ''))
                except:
                    info[topic_info] = detail_value[0].replace('원', '').replace(',', '')

        product_data.append(info)
        cnt += 1

In [5]:
import pandas as pd 

df = pd.DataFrame(product_data)
df = df.drop_duplicates(subset=['상품분류', '상품명', '원가', '할인율', '상품설명', '원산지', '구매혜택', '배송 방법', '배송비'])
# 1) 상품명 중복만 추출
dup_names = df['상품명'].value_counts()
dup_names = dup_names[dup_names >= 2].index  # 중복된 상품명 목록

# 2) 중복 상품명 AND '세일&이벤트' 조건인 행만 삭제
condition = (df['상품명'].isin(dup_names)) & (df['상품분류'] == '세일&이벤트')
df = df.drop(df[condition].index)

df.reset_index(drop=True, inplace=True)
df.head()

,상품ID,상품분류,상품명,원가,할인율,상품설명,원산지,구매혜택,배송 방법,배송비
0,1,세일&이벤트,[새턴버드 x 네이버페이] 콜드브루 커피 6개입 (2종 혼합),14000,0,새턴버드 커피는 독자적인 신기술인 DIAMONDUST™ 공법으로 새롭게 탄생한 커피...,중국,450,택배,3000
1,2,세일&이벤트,[새턴버드 x 헬리녹스] 콜드브루 커피 6개입 (리유저블 컵 포함),16000,6,새턴버드 커피는 독자적인 신기술인 DIAMONDUST™ 공법으로 새롭게 탄생한 커피...,중국,450,택배,3000
2,3,세일&이벤트,[새턴버드] 익스프레스 콜드브루 커피 스타터 키트 (익스프레스 2종 + 텀블러),68000,10,새턴버드 커피는 독자적인 ZEROLOSS™ 동결건조 방식으로 신선하게 볶은 최고급 ...,중국,450,택배,3000
3,81,선물세트,[새턴버드] 넘버 시리즈 콜드브루 커피 12개입 선물세트 (6종 혼합),29000,0,새턴버드 커피는 독자적인 신기술인 DIAMONDUST™ 공법으로 새롭게 탄생한 스페...,중국,450,택배,3000
4,82,선물세트,"[새턴버드] 넘버 시리즈 콜드브루 커피 선물세트 (구성 택2) (37,000원 ~ ...",37000,0,새턴버드 커피는 독자적인 신기술인 DIAMONDUST™ 공법으로 새롭게 탄생한 스페...,중국,450,택배,3000


In [11]:
df['상품ID'] = list(range(1,33))
df.columns = [i.replace(' ','') for i in df.columns]
df = df.rename(columns={'구매혜택' : '구매혜택포인트'})
df.to_csv('데이터_상품테이블.csv',index=False)

In [1]:
import pandas as pd
import random

# =====================
# 설정값
# =====================
INPUT_CSV = "데이터_상품테이블.csv"
OUTPUT_CSV = "데이터_상품테이블_50000.csv"
TARGET_COUNT = 50000

# =====================
# 원본 상품 로드 (32건)
# =====================
base_df = pd.read_csv(INPUT_CSV)

base_count = len(base_df)
repeat_factor = TARGET_COUNT // base_count + 1

# =====================
# 상품 복제
# =====================
df = pd.concat([base_df] * repeat_factor, ignore_index=True)
df = df.iloc[:TARGET_COUNT].copy()

# =====================
# 상품ID 재생성
# =====================
df["상품ID"] = range(1, TARGET_COUNT + 1)

# =====================
# 가격 / 할인율 / 포인트 분포 설계
# =====================
def adjust_price(base_price):
    # ±20% 변동
    ratio = random.uniform(0.8, 1.2)
    return int(round(base_price * ratio / 100)) * 100

def random_discount():
    # 할인율 분포
    r = random.random()
    if r < 0.7:
        return 0
    elif r < 0.9:
        return random.choice([5, 10])
    else:
        return random.choice([20, 30, 40])

def random_point(price):
    return int(price * random.uniform(0.02, 0.05))

def random_delivery_fee():
    return random.choice([0, 3000, 5000])

# =====================
# 컬럼 변형
# =====================
df["원가"] = df["원가"].apply(adjust_price)
df["할인율"] = df["할인율"].apply(lambda _: random_discount())
df["구매혜택포인트"] = df["원가"].apply(random_point)
df["배송비"] = df["배송비"].apply(lambda _: random_delivery_fee())

# =====================
# 상품명에 변형 태그 추가 (중복 회피)
# =====================
df["상품명"] = df.apply(
    lambda r: f"{r['상품명']} (옵션{r.name % 5 + 1})",
    axis=1
)

# =====================
# CSV 저장
# =====================
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print(f"상품 {len(df):,}건 생성 완료 → {OUTPUT_CSV}")

상품 50,000건 생성 완료 → 데이터_상품테이블_50000.csv
